# Task: Text Generation

In this Notebook we run Archai's [Text Generation](https://github.com/microsoft/archai/tree/main/tasks/text_generation) task on Azure Machine Learning.

Our goal is to show how to create and run jobs for each step of the task without spending lots of computing resources. Therefore, our goal is not to train a good model -- for this purpose please refer to the original task.

## Prerequisites

- Python 3.7 or later
- An Azure subscription
- An Azure Resource Group
- An Azure Machine Learning [Workspace](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources#create-the-workspace)

### Requirements

In [ ]:
%pip install azure-ai-ml azure-identity 
%pip install jinja2
%pip install archai

In [ ]:
import os
from pathlib import Path

from IPython.display import display, Image
from IPython.core.display import HTML

from azure.ai.ml import load_job

import archai.common.azureml_helper as aml_helper
import archai.common.notebook_helper as nb_helper

### Get a handle to the workspace

We load the workspace from a workspace [configuration file](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-environment#local-and-dsvm-only-create-a-workspace-configuration-file).

In [ ]:
ml_client = aml_helper.get_aml_client_from_file("../.azureml/config.json")
print(f'Using workspace: {ml_client.workspace_name} in resource group: {ml_client.resource_group_name}')

### Create a CPU compute cluster

We provision a Linux [compute cluster](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python) for the NAS job in this Notebook. See the [full list](https://azure.microsoft.com/en-ca/pricing/details/machine-learning/) on VM sizes and prices.

In [ ]:
cpu_compute_name = "nas-cpu-cluster-D14-v2"
cpu_compute_cluster = aml_helper.create_compute_cluster(ml_client, cpu_compute_name, size="Standard_D14_v2")

### Create a GPU compute cluster

For full training we provision a GPU compute cluster.

In [ ]:
gpu_compute_name = "nas-gpu-cluster-NC6"
gpu_compute_cluster = aml_helper.create_compute_cluster(ml_client, gpu_compute_name, size="Standard_NC6")

### Create an environment based on a YAML file

Azure Machine Learning maintains a set of CPU and GPU Ubuntu Linux-based base images with common system dependencies. For the set of base images and their corresponding Dockerfiles, see the [AzureML Containers](https://github.com/Azure/AzureML-Containers) repo.

In [ ]:
archai_job_env = aml_helper.create_environment_from_file(ml_client, image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest", conda_file="conda.yaml")

### Job 1: NAS (Searching for Pareto-optimal Architectures)

#### Load the search job from a YAML file and run it

In [ ]:
search_job = load_job(source=os.path.join("src", "search.yaml"))
s_job = ml_client.create_or_update(search_job)

#### Stream logs of the job

This cell will automatically complete when the job is finished.

In [ ]:
ml_client.jobs.stream(s_job.name)

#### Download job's output

In [ ]:
output_name = "output_dir"
download_path = "output"

aml_helper.download_job_output(ml_client, job_name=s_job.name, output_name=output_name, download_path=download_path)

downloaded_folder = Path(download_path) / "named-outputs" / output_name

#### Show Pareto Frontiers

In [ ]:
param_vs_latency_img = Image(filename=downloaded_folder / "pareto_non_embedding_params_vs_onnx_latency.png")
display(param_vs_latency_img)

In [ ]:
param_vs_memory_img = Image(filename=downloaded_folder / "pareto_non_embedding_params_vs_onnx_memory.png")
display(param_vs_memory_img)

In [ ]:
latency_vs_memory_img = Image(filename=downloaded_folder / "pareto_onnx_latency_vs_onnx_memory.png")
display(latency_vs_memory_img)

#### Show search state of the last iteration

In [ ]:
df = nb_helper.get_search_csv(downloaded_folder)
csv_as_html = nb_helper.get_csv_as_stylized_html(df)
display(HTML(csv_as_html))

### Job 2: Train (Train a Pareto architecture from Transformer-Flex.)

Pick an architecture id (archid) from the CSV file to perform full training on

In [ ]:
archid = "<arch-id>"
arch_path = nb_helper.get_arch_abs_path(archid=archid, downloaded_folder=downloaded_folder)

Load the training job from a YAML file and set the arch_path as its input

In [ ]:
train_job = load_job(source=os.path.join("src", "train.yaml"))
train_job.inputs.arch_config_path.path = arch_path

In [ ]:
t_job = ml_client.create_or_update(train_job)

#### Stream logs of the job

This cell will automatically complete when the job is finished.

In [ ]:
ml_client.jobs.stream(t_job.name)

### Job 3: Generating text via prompt

Load the generate text job from a YAML file, set the trained model path, and run it

In [ ]:
gen_job = load_job(source=os.path.join("src", "generate_text.yaml"))
gen_job.inputs.pre_trained_model_path.path = "azureml://full/path/to/trained/model/checkpoint"

g_job = ml_client.create_or_update(gen_job)
ml_client.jobs.stream(g_job.name)

#### Download and show generated text

In [ ]:
output_name = "output_path"
download_path = "generated_text"

aml_helper.download_job_output(ml_client, job_name=g_job.name, output_name=output_name, download_path=download_path)

downloaded_file = Path(download_path) / "named-outputs" / output_name / output_name
with open(downloaded_file, "r") as f:
    print(f.read())